In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon, box
from scipy.interpolate import griddata
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

In [2]:
# load the Swiss Glacier Inventory 2016
sgi = gpd.read_file('data/sgi_2016/SGI_2016_glaciers.shp')

### 1. Loading and investigating a vector dataset with geopandas
1) What datatype did you just load?
2) How many glaciers are contained in the dataset?
3) How many attributes does the dataset have and which ones?
4) What are the datatypes of the individual columns?
5) Which is the largest glacier, which one is the smallest glacier?
6) How many glaciers have a name?

Try using the attributes & functions gdf.shape, gdf.columns, gdf.dtypes, gdf.idxmin() and others to do this. Try using the attributes & functions gdf.shape, gdf.columns, gdf.dtypes, gdf.idxmin() and others to do this. Replace the three dots with the right commands. 

In [3]:
sgi.head()

,gid,pk_glacier,sgi-id,name,rl_0,rl_1,rl_2,rl_3,i_code,year_acq,year_rel,area_km2,length_km,masl_min,masl_med,masl_mean,masl_max,slope_deg,aspect_deg,geometry
0,16559.0,bb1e06de-74a9-11ea-bc55-0242ac130003,A10g-04,None,A,1,0,g,04,2014,2020,0.015169,0.234,3033,3101,3101,3168,32.99,268,"POLYGON ((2802224.865 1193097.586, 2802227.799..."
1,1503.0,80f6be00-4ec8-11e8-85b0-985fd331b2ee,A54e-12,Steigletscher,A,5,4,e,12,2016,2020,5.561256,4.234,2032,2932,2877,3494,20.13,342,"MULTIPOLYGON (((2676375.305 1174502.297, 26763..."
2,17178.0,80e3ab30-4ec8-11e8-9357-985fd331b2ee,A54e-19,Vorder Tierberg (Innertkirchen),A,5,4,e,19,2016,2020,0.578672,1.545,2315,2700,2681,3070,25.43,261,"POLYGON ((2673410.261 1172337.403, 2673426.795..."
3,16556.0,8211f200-4ec8-11e8-9016-985fd331b2ee,A10g-09,Chlein Wintertälli,A,1,0,g,09,2014,2020,0.077443,0.551,2634,2753,2749,2847,28.38,343,"POLYGON ((2798559.905 1190687.342, 2798585.701..."
4,16777.0,809e16ae-4ec8-11e8-a93d-985fd331b2ee,A54j-01,Ochsental,A,5,4,j,01,2016,2020,0.028602,0.322,2118,2191,2191,2268,30.15,332,"POLYGON ((2656707.845 1169768.145, 2656717.484..."


In [4]:
# what datatype is the data you just loaded?
type(sgi)

geopandas.geodataframe.GeoDataFrame

In [5]:
# how many rows and colums does it have?
sgi.shape #number of (rows, colums)

(1400, 20)

In [6]:
# what attributes does it have?
sgi.columns #attributes

Index(['gid', 'pk_glacier', 'sgi-id', 'name', 'rl_0', 'rl_1', 'rl_2', 'rl_3',
       'i_code', 'year_acq', 'year_rel', 'area_km2', 'length_km', 'masl_min',
       'masl_med', 'masl_mean', 'masl_max', 'slope_deg', 'aspect_deg',
       'geometry'],
      dtype='object')

In [12]:
# What are the datatypes of the different attributes?
sgi.dtypes #datatypes

gid            float64
pk_glacier      object
sgi-id          object
name            object
rl_0            object
rl_1             int64
rl_2             int64
rl_3            object
i_code          object
year_acq         int64
year_rel         int64
area_km2       float64
length_km      float64
masl_min         int64
masl_med         int64
masl_mean        int64
masl_max         int64
slope_deg      float64
aspect_deg       int64
geometry      geometry
dtype: object

In [8]:
# How is the geometry stored as a whole?
type(sgi.geometry)

geopandas.geoseries.GeoSeries

In [9]:
# What do these geometries look like, and what datatype are they?
# How is the geometry stored as a whole?
print(sgi.geometry)
print(type(sgi.geometry[0]))

0       POLYGON ((2802224.865 1193097.586, 2802227.799...
1       MULTIPOLYGON (((2676375.305 1174502.297, 26763...
2       POLYGON ((2673410.261 1172337.403, 2673426.795...
3       POLYGON ((2798559.905 1190687.342, 2798585.701...
4       POLYGON ((2656707.845 1169768.145, 2656717.484...
                              ...                        
1395    POLYGON ((2772945.740 1133624.726, 2772947.490...
1396    MULTIPOLYGON (((2672856.354 1174156.594, 26728...
1397    POLYGON ((2672857.466 1173500.634, 2672854.581...
1398    MULTIPOLYGON (((2673349.861 1172347.237, 26733...
1399    POLYGON ((2802438.840 1192171.652, 2802402.284...
Name: geometry, Length: 1400, dtype: geometry
<class 'shapely.geometry.polygon.Polygon'>


In [10]:
# Which glacier is largest, which one smallest?
max_area_id = sgi['area_km2'].idxmin()
sgi.loc[max_area_id]

gid                                                     11761.0
pk_glacier                 8028e6b0-4ec8-11e8-9a5f-985fd331b2ee
sgi-id                                                   B35-10
name                                            Unterbächhorn W
rl_0                                                          B
rl_1                                                          3
rl_2                                                          5
rl_3                                                       None
i_code                                                       10
year_acq                                                   2017
year_rel                                                   2020
area_km2                                               0.010095
length_km                                                   0.0
masl_min                                                   3163
masl_med                                                   3210
masl_mean                               

In [13]:
# How many glaciers don't have a name? 
sgi['name'].isnull().sum() #number of glaciers with no name (or with name --> .notnull())

138

### 2. Let's look at the spatial part in a bit more detail

Select one glacier and investigate it's geometry in more detail answering the following questions:

1) What datatype does the geometry have / how is it stored?
2) Explore the geometry's attributes length, area, centroid and bounds. What can you learn?
3) What coordinate reference system is the dataset in?
4) What happens if you query length, area etc. on the whole SGI?

In [14]:
# Make a new variable that contains the geometry of only one glacier of your choice. 
myglacier = sgi[sgi['name']=='Silvrettagletscher']

In [15]:
myglacier.length #How is this length different from the length_km stored in the sgi dataset?

1399    10799.938896
dtype: float64

In [16]:
myglacier.crs

<Projected CRS: EPSG:2056>
Name: CH1903+ / LV95
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Liechtenstein; Switzerland.
- bounds: (5.96, 45.82, 10.49, 47.81)
Coordinate Operation:
- name: Swiss Oblique Mercator 1995
- method: Hotine Oblique Mercator (variant B)
Datum: CH1903+
- Ellipsoid: Bessel 1841
- Prime Meridian: Greenwich

In [17]:
sgi.centroid

0       POINT (2802161.070 1193055.473)
1       POINT (2676113.725 1172285.380)
2       POINT (2672820.701 1171851.255)
3       POINT (2798638.320 1190524.736)
4       POINT (2656721.432 1169662.692)
                     ...               
1395    POINT (2772910.852 1133539.729)
1396    POINT (2672693.070 1173875.746)
1397    POINT (2672684.077 1173439.230)
1398    POINT (2673860.931 1172451.055)
1399    POINT (2801195.406 1192405.841)
Length: 1400, dtype: geometry

### 3. Plotting

1) Use sgi.plot() to plot the Swiss Glacier Inventory. Ok, nice, but not particularly useful (yet). 
2) Zoom in (set x and y limits) on the area around Silvretta glacier.
3) Find the right bounds for the chloropleth map that shows a good category for the different glacier areas.

In [ ]:
sgi.plot()

In [ ]:
def meters_to_kilometers(x, pos):
    """Convert meters to kilometers."""
    return f'{x / 1e3:.0f}'

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))
sgi.plot(ax=ax, 
         column='area_km2', 
         vmin=0.03, 
         vmax=3, 
         legend=True,
         legend_kwds={"label": "Glacier size", "orientation": "horizontal"})
ax.set_xlim([2795000, 2811000])
ax.set_ylim([1186250, 1195000])

# Create custom formatters for the x and y axis
x_formatter = FuncFormatter(meters_to_kilometers)
y_formatter = FuncFormatter(meters_to_kilometers)

# Apply the formatters to the axes for pretty labels
ax.xaxis.set_major_formatter(x_formatter)
ax.yaxis.set_major_formatter(y_formatter)

ax.set_xlabel('Easting (km)')
ax.set_ylabel('Northing (km)')
ax.set_title('Silvretta-region glacier areas', fontsize=15)

When you like your result, save it as a pdf. PDF is a useful format because it can be imported into vector editing programs (Adobe Illustrator, Affinity Designer, Inkscape...) for final editing.

In [ ]:
#fig.savefig('Silvretta_glaciers.pdf')

### 4. Filtering spatially

1) Find all glaciers withing the Silvretta region (use the bounding box you defined above)
2) Find all glaciers that are in 

In [ ]:
# Example bounding box coordinates (use the ones you defined above)
minx, miny, maxx, maxy = [2795000, 1186250, 2811000, 1195000]  # Replace with your bounding box coordinates

In [ ]:
# Create a bounding box
silvretta_region = box(minx, miny, maxx, maxy)

In [ ]:
silvretta_glaciers = sgi[sgi.geometry.within(silvretta_region)]

In [ ]:
# Find all glaciers that are in southern Switzerland (e.g., below y<1 130 000)
southern = sgi.cx[ : , :1130000].plot() 

In [ ]:
# Now adapt this for eastern, western or northern Switzerland (not strictly sticking to typical conventions for the regions)
eastern = sgi.cx[ 2650000: , : ].plot() 
werstern = sgi.cx[ : 2650000, : ].plot() 
northern = southern = sgi.cx[ : ,1130000 :].plot() 

### 5. Some typical geometric operations

Geopandas offers functionality for typical geometric operations such as smoothing or buffering. Check the documentation to see what arguments [geopandas.GeoSeries.simplify](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.simplify.html) and
[geopandas.GeoSeries.buffer()](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoSeries.buffer.html) take.

Create 
1) a simplified version of your glacier outline and plot both the original and smoothed version
2) create a buffer around your glacier and plot both the buffered and the original outlines

In [ ]:
# smooth
silvretta_smoothed = sgi[sgi.name=='Silvrettagletscher'].geometry.simplify(25)

In [ ]:
f, ax = plt.subplots()
silvretta_smoothed.plot(ax=ax, facecolor='none', edgecolor='k')
sgi[sgi.name=='Silvrettagletscher'].geometry.plot(ax=ax, facecolor='none', edgecolor='red')

In [ ]:
# buffer
silvretta_buffered = sgi[sgi.name=='Silvrettagletscher'].geometry.buffer(50)

In [ ]:
f, ax = plt.subplots()
silvretta_buffered.plot(ax=ax, facecolor='none', edgecolor='k')
sgi[sgi.name=='Silvrettagletscher'].geometry.plot(ax=ax, facecolor='none', edgecolor='red')

### 6. Writing spatial data with pandas and geopandas

We want to be able to query our spatial datasets at defined locations! 
Create and save a shape file that contains three sampling points within Silvretta glacier by follwoing these steps:
1) define points
2) turn the points into a python dictionary and use the shapely Point() function to turn your points into shaply geometry objects.
3) turn your dictionary into a Pandas GeoDataFrame
4) save the gdf to file using "GeoJSON" as the driver.
5) inspect the file you just saved in using a text editor. What advantages do you see in comparison to the traditional .shp files?
6) Let's also save the outline of Silvretta glacier.
7) Want to sample randomly? Investigate the function geopandas.GeoSeries.sample_points()

In [3]:
# create geojson file with points of interest for sampling
point1 = 2800379.7, 1192725.2
point2 = 2801251.5, 1192437.9
point3 = 2802045.6, 1191907.4

In [4]:
# turn points into dictionary
sps = {'Points': ['Point1', 'Point2', 'Point3'], 'geometry': [Point(point1), Point(point2), Point(point3)]}
# turn dictionary into GeoDataFrame
sampling_points = gpd.GeoDataFrame(sps, crs='EPSG:2056')

In [6]:
sampling_points.to_file('./data/sampling_points.geojson', driver='GeoJSON')

In [7]:
# Let's also save our outline for re-use in further edits
silv_outline = sgi[sgi.name=='Silvrettagletscher']
silv_outline.geometry.to_file('./data/silvretta_outline.geojson', driver='GeoJSON')

In [ ]:
# What if you wanted to sample a random set of points within your outline?
rand_points = silv_outline.sample_points(150)
rand_points.plot()

### 7. Let's add some spatial data that is available in tabular form

Load the Glacier Ice Thickness database (glathida) and inspect the dataset.

1) What attributes does it have?
2) What spatial coordinate system do you think this is in? Does the dataset know this already?
3) Go ahead and extract all the ice thickness measurements for Silvrettaglacier into its own variable.
4) Turn the dataframe into a spatially-aware GeoDataFrame that allows you to perform spatial operations and reproject it into the Swiss Coordinate System CH1903+ / LV95 

In [10]:
# Load ice thickness measurements
glathida = pd.read_csv('data/glathida-3.1.0/data/TTT.csv', low_memory=False)

In [ ]:
# Inspect the dataset
glathida.head()

In [11]:
# Extract the data for Silvretta glacier
silvretta = glathida[glathida.GLACIER_NAME=='SILVRETTA']

In [12]:
# make into geopandas dataframe
geometry = [Point(xy) for xy in zip(silvretta['POINT_LON'], silvretta['POINT_LAT'])] #list-comprehension & zip() iterator on tuples of coordinates
silvretta_gdf = gpd.GeoDataFrame(silvretta, geometry=geometry)

In [13]:
# explore the crs
print("Current CRS:", silvretta_gdf.crs)

Current CRS: None


In [14]:
silvretta_gdf = silvretta_gdf.set_crs(epsg=4326, inplace=True)

In [15]:
# reproject to Swiss Coordinate System LV95
silvretta_lv95 = silvretta_gdf.to_crs(epsg=2056)

### 8. Spatial operations with point data

We now have two spatial datasets from the same area (glacier outlines and ice thickness measurements). Let's explore how the two can be used together. 


1) Create a new variable that cointains all the ice thickness points that are inside our glacier outline using [gpd.sjoin()](https://geopandas.org/en/stable/docs/reference/api/geopandas.sjoin.html)
2) Plot outline and all the points, marking the ones that are inside the outline.
3) In a second figure, color the points by their ice thickness values
4) How would you go about ensuring that the points you consider "inside" are at a minimum distance from the outline?

In [16]:
# Find the points that are inside the polygon
# Step 1: Spatial Join
points_inside = gpd.sjoin(silvretta_lv95, silv_outline, predicate='within')

In [21]:
# Let's save these for future use!
points_inside.to_file('data/glathida_inside_sgi.geojson', driver='GeoJSON')

In [ ]:
fig, ax = plt.subplots(figsize=(10, 10))
silvretta_lv95.plot(ax=ax, marker='o', color='orange', markersize=20, alpha=0.5, zorder=1)
points_inside.plot(ax=ax, marker='o', color='k', markersize=20, alpha=0.25, zorder=2)
silv_outline.plot(ax=ax, color='steelblue', zorder=0)
plt.title('Silvretta GPR')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

In [ ]:
# using an inside buffer
silv_outline_buffered = silv_outline.geometry.buffer(-25) #buffer inside
silv_outline.loc[:,['buffered']] = silv_outline_buffered #add geometry as new column (could also replace original one)
silv_outline = silv_outline.set_geometry('buffered') #set new column as geometry
silv_outline = silv_outline.drop('geometry', axis=1) # if added as new column, drop old one, otherwise spatial join keeps geomety, which screws up the plotting
points_inside_buff = gpd.sjoin(silvretta_lv95, silv_outline, predicate='within') # re-run spatial join

In [ ]:
# Now lets look at the ice thickness values for the points inside our outline
fig, ax = plt.subplots(figsize=(10, 10))
d = plt.scatter(
    points_inside.geometry.x,  # x-coordinates of points
    points_inside.geometry.y,  # y-coordinates of points
    c=points_inside['THICKNESS'],  # values to use for coloring
    cmap='Blues',  # colormap
    s=50,  # size of markers
    edgecolor='none',  # edge color of markers
    alpha=0.8  # transparency
)
silv_outline.plot(ax=ax, edgecolor='k', facecolor='none', zorder=0)
cbar = fig.colorbar(d, ax=ax, label='Thickness', shrink=0.5)
plt.title('Silvretta GPR')
plt.xlabel('x')
plt.ylabel('y')
plt.show()

### 9. Let's interpolate these points 

In [ ]:
# Extract the bounds of the polygon
minx, miny, maxx, maxy = silv_outline.total_bounds
# Create an empty grid for interpolation
grid_x, grid_y = np.mgrid[minx:maxx:100j, miny:maxy:100j]  # 100j for a 100x100 grid

In [ ]:
points = np.array(list(zip(points_inside.geometry.x, points_inside.geometry.y)))
values = points_inside['THICKNESS'].values

In [ ]:
grid_z = griddata(points, values, (grid_x, grid_y), method='linear') #try also cubic. Which one looks better?

In [ ]:
f, ax = plt.subplots()
#using imshow
ax.imshow(grid_z.T, origin='lower', extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()], aspect='auto', cmap='Blues')
#using contour
#ax.contourf(grid_x, grid_y, grid_z, cmap='Blues')

#add points from measurements
d = plt.scatter(
    points_inside.geometry.x,  # x-coordinates of points
    points_inside.geometry.y,  # y-coordinates of points
    c=points_inside['THICKNESS'],  # values to use for coloring
    cmap='Blues',  # colormap
    s=50,  # size of markers
    edgecolor='none',  # edge color of markers
    alpha=1  # transparency
)
silv_outline.plot(ax=ax, facecolor='None')

In [ ]:
### Rasterize and fill interpolation
#This doesn't really work... Cut losses and export as geotiff to work on with raster data?


In [ ]:
xv, yv = np.meshgrid(np.linspace(minx, maxx, 100), np.linspace(miny, maxy, 100))
grid_points = np.c_[xv.ravel(), yv.ravel()]
mask = np.zeros_like(grid_z, dtype=bool)

In [ ]:
# Use shapely to create a mask
for geom in silv_outline.geometry:
    mask |= np.array([geom.contains(Point(x, y)) for x, y in grid_points]).reshape(grid_z.shape)

In [ ]:
# Set z-values to zero along the outline
grid_z[~mask] = 0

In [ ]:
# Combine original grid points with the outline points
outline_points = np.array(silv_outline.iloc[0].geometry.exterior.coords)
all_points = np.vstack([grid_points, outline_points])
all_values = np.hstack([grid_z.ravel(), np.zeros(len(outline_points))])

In [ ]:
grid_z_extrapolated = griddata(all_points, all_values, (grid_x, grid_y), method='nearest')

In [ ]:
f, ax = plt.subplots()
#using imshow
ax.imshow(grid_z_extrapolated, origin='lower', extent=[grid_x.min(), grid_x.max(), grid_y.min(), grid_y.max()], aspect='auto', cmap='Blues')

#add points from measurements
d = plt.scatter(
    points_inside.geometry.x,  # x-coordinates of points
    points_inside.geometry.y,  # y-coordinates of points
    c=points_inside['THICKNESS'],  # values to use for coloring
    cmap='Blues',  # colormap
    s=50,  # size of markers
    edgecolor='none',  # edge color of markers
    alpha=1  # transparency
)
silv_outline.plot(ax=ax, facecolor='None')